In [340]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import os
import random
import math

In [389]:
def HurstFunction(rawBOLD, TR):
    '''This function will analyze a rs-BOLD raw signal and output 
    the appropriate Hurst exponent according to the procedure of 
    Eke et al. (Eur J Physiol (2000) 439:403?415)'''
    
    #initialize variables
    Hurst = None
    abort = False
    
    #common parameters
    fs = 1/TR #sampling frequency
    print('Sampling frequency =', fs)
    n = len(rawBOLD) #number of timepoints
    print('Number of timepoints =', n)

    #normalize the time series
    mean_rawBOLD = np.mean(rawBOLD)
    rawBOLD_norm = rawBOLD - mean_rawBOLD #rawBOLD normalized by subtracting mean from every data point
    print('Mean rawBOLD =', mean_rawBOLD)
    
   #determine the parabolic window (W) of the series
    N = len(rawBOLD_norm)
    print(N)
    W = np.zeros((N,1))
    for j in np.arange(0,N):
        W[j] = 1 - np.power((2*(j + 1)/(N+1)-1),2)

    #multiply each normalized value by parabolic window to get parabolic window signal
    signal_pw = np.array(rawBOLD_norm) * np.array(W)
    
    #end matching of signal_pw
    #line equation is line connecting first and last points of signal_pw
    y_first = signal_pw[0,0] 
    y_last = signal_pw[-1,-1]
    slope_ends = (y_last - y_first)/(N-1)
    y_int = y_last - slope_ends*N
    x_values = np.arange(1,N+1)
    line_eq = slope_ends * x_values + y_int
    
    
    #bridge detrending
    #subtract transposed line equation from signal_pw
    signal_em1 = signal_pw - line_eq[:, np.newaxis]
    
    #range is first half of signal
    range = math.ceil((N+1)/2)
    
    #frequency vector is sample freq times (0-range)/N
    freq_row_vector = fs * np.arange(0,range)/N
    #transpose freq_row_vector to get freq in column format
    freq = freq_row_vector[:,np.newaxis]
    #signal_em1 is the signal after lowPSDwe has been applied, use this signal for PSD

    return Hurst


In [392]:
#basic function call to test print statements
HurstFunction([1,4,2], 2)

Sampling frequency = 0.5
Number of timepoints = 3
Mean rawBOLD = 2.3333333333333335
3


In [393]:
#run this cell to test Hurst function 
#cell loads data files and makes function call 

#find nifti files to test function with
dir = os.path.join(os.getcwd(), 'sub-08_ses-1/func')
file_csf = os.path.join(dir, 'csf_mask.nii.gz')
file_grey = os.path.join(dir, 'grey_mask.nii.gz')
file_white = os.path.join(dir, 'white_mask.nii.gz')
file_noise = os.path.join(dir, 'noise_mask.nii.gz')
file_sub = os.path.join(dir, 'sub-08_ses-1_task-ins_bold.nii.gz')

#load files into notebook
csfmask = nib.load(file_csf).get_fdata()
csflog = csfmask.astype(bool)
greymask = nib.load(file_grey).get_fdata()
greylog = greymask.astype(bool)
whitemask = nib.load(file_white).get_fdata()
whitelog = whitemask.astype(bool)
noisemask = nib.load(file_noise).get_fdata()
noiselog = noisemask.astype(bool)

sub8ins = nib.load(file_sub).get_fdata()

#find random index of array where element is true after applying mask
[row_c,col_c,page_c] = random.choice(np.argwhere(csflog))
[row_g, col_g, page_g] = random.choice(np.argwhere(greylog))
[row_w,col_w,page_w] = random.choice(np.argwhere(whitelog))
[row_n,col_n,page_n] = random.choice(np.argwhere(noiselog))

#get element in sub8ins data at random index
csf_sample = np.squeeze(sub8ins[row_c,col_c,page_c,:])
grey_sample = np.squeeze(sub8ins[row_g,col_g,page_g,:])
white_sample = np.squeeze(sub8ins[row_w,col_w,page_w,:])
noise_sample = np.squeeze(sub8ins[row_n,col_n,page_n,:])


#call to Hurst function
HurstFunction(noise_sample,2)




Sampling frequency = 0.5
Number of timepoints = 176
Mean rawBOLD = 82.99431818181819
176
